# 1. Importing some Libraries

In [ ]:
import numpy as np
import pandas as pd
from influxdb import InfluxDBClient
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import plotly.graph_objects as go
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt

import holoviews as hv
from holoviews import opts
hv.extension('bokeh')


from tensorflow.keras.layers import Input, Dropout, Dense, LSTM, TimeDistributed, RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from sklearn import metrics
import tensorflow as tf
import keras.backend as K
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 15)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)


# 2. Fetching Data: DE2Thing_HMD8310

In [ ]:
# Connect to InfluxDB and fetch data
client = InfluxDBClient(host='localhost', port=8086)
client.switch_database('ISS')

# Query to the Database for one measurement

#query1 = 'SELECT * FROM "DE2Thing_HMD8310"'
query1 = 'SELECT * FROM "DE2Thing_HMD8310" WHERE time >= \'2022-08-29T23:28:00Z\' AND time < \'2023-04-26T08:00:00Z\''
#query1 = 'SELECT * FROM "DE1Thing_HMD8310" WHERE time >= \'2022-09-30T00:00:00Z\' AND time < \'2022-10-30T23:59:00Z\''
results1 = client.query(query1)
DE2Thing_HMD8310 = pd.DataFrame.from_records(results1.get_points())

print(DE2Thing_HMD8310.shape)
DE2Thing_HMD8310.head(2)

In [ ]:
print(DE2Thing_HMD8310.columns.to_list())

In [ ]:
DE2Thing_HMD8310.to_csv("Data/DE2Thing_Load_Power.csv")

In [ ]:
# Function to convert ISO 8601 time to datetime
def ISO_8601_To_Datetime(s):
    return datetime.strptime(s, '%Y-%m-%dT%H:%M:%SZ')

In [ ]:
# Read The data
df = pd.read_csv('Data/DE2Thing_Load_Power.csv', parse_dates=[1], index_col=0, date_parser=ISO_8601_To_Datetime)
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
# Select 3 Features :
df_Load_Power = df.loc[:, ['time', 'Load','Power']]

print(df_Load_Power.shape)
df_Load_Power.head(2)

In [ ]:
df_Load_Power.isnull().sum()

In [ ]:
df_Load_Power.dropna()

In [ ]:
df_Load_Power.head(4)

In [ ]:
df_Load_Power = df_Load_Power.dropna()
df_Load_Power

In [ ]:
df_Load_Power.isnull().sum()

In [ ]:
#df_Load_Power = df_Load_Power.fillna(df_Load_Power.mean())

In [ ]:
# we want to replace the zeros in column 'Load' with the mean of 'Load'.
print("The Mean is:", df_Load_Power['Load'].mean())
mean_value_L = df_Load_Power['Load'].mean()
df_Load_Power['Load'].replace(0, mean_value_L, inplace=True)
df_Load_Power

In [ ]:
# we want to replace the zeros in column 'Power' with the mean of 'Power'.
print("The Mean is:", df_Load_Power['Power'].mean())
mean_value_P = df_Load_Power['Power'].mean()
df_Load_Power['Power'].replace(0, mean_value_P, inplace=True)
df_Load_Power

In [ ]:
df_Load_Power

In [ ]:
# Checking for blank values and Data Types.
def overview(df_Load_Power: pd.DataFrame, timestamp_col: str= None) -> None:
    print('Null Count:\n', df_Load_Power.isnull().sum(), '\n')
    print('Data Types:\n:', df_Load_Power.dtypes)
    
    if timestamp_col is not None:
        print('\nDate Range: \n\nStart:\t', df_Load_Power[timestamp_col].min())
        print('End:\t', df_Load_Power[timestamp_col].max())
        print('Days:\t',(df_Load_Power[timestamp_col].max() - df_Load_Power[timestamp_col].min()))

In [ ]:
overview(df_Load_Power, timestamp_col='time')

In [ ]:
df_Load_Power

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_Load_Power["time"], y=df_Load_Power["Load"], mode='lines', name='Load'))
fig.update_layout(title_text="Load", yaxis1=dict(title="Load", side='left'))

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_Load_Power["time"], y=df_Load_Power["Power"], mode='lines', name='Power'))
fig.update_layout(title_text="Power", yaxis1=dict(title="Power", side='left'))

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_Load_Power["time"], y=df_Load_Power["Load"], mode='lines', name='Load'))
fig.add_trace(go.Scatter(x=df_Load_Power["time"], y=df_Load_Power["Power"], mode='lines', name='Power', yaxis='y2'))
fig.update_layout(title_text="Load vs Power",
                  yaxis1=dict(title="Load", side='left'),
                  yaxis2=dict(title="Power", side='right', anchor="x", overlaying="y")
                  )

fig.show()

In [ ]:
df_Load_Power.head(4)

In [ ]:
df_Load_Power.info()

# LSTM Autoencoder

## Split The Data into Train and Test Set

In [ ]:
#df = df.loc[:2000,:]
df_Load_Power = df_Load_Power.loc[:, ['time','Load']]
df_timestamp = df_Load_Power[['time']]

df_ = df_Load_Power[['Load']]
df_Load_Power.shape


In [ ]:
train_prp = .98
train = df_.loc[:df_.shape[0] * train_prp]
test = df_.loc[df_.shape[0] * train_prp:]

In [ ]:
train

In [ ]:
test

## Feature Scaling

In [ ]:
# Standardize The Data
scaler = StandardScaler()
X_train = scaler.fit_transform(train)
X_test = scaler.transform(test)


print("X train Shape:", X_train.shape)
print("X test Shape:", X_test.shape)

In [ ]:
X_train

In [ ]:
# Reshape the Dimension of the Train and Test set for LSTM Model
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

print("X train Shape:", X_train.shape)
print("X test Shape:", X_test.shape)

In [ ]:
def autoencoder_model(X):
    # The Encoder
    inputs = Input(shape=(X.shape[1],  X.shape[2]))
    L1 = LSTM(16, activation='relu', return_sequences=True, kernel_regularizer=regularizers.l2(0.00))(inputs)
    L2 = LSTM(4, activation='relu', return_sequences=False)(L1)
    
    L3 = RepeatVector(X.shape[1])(L2)
    
    # The Decoder
    L4 = LSTM(4, activation='relu', return_sequences=True)(L3)
    L5 = LSTM(16, activation='relu', return_sequences=True)(L4)
    output = TimeDistributed(Dense(X.shape[2]))(L5)
    model = Model(inputs=inputs, outputs=output)
    return model

In [ ]:
model = autoencoder_model(X_train)
model.compile(optimizer='adam', loss='mae', metrics=['accuracy'])
model.summary()

In [ ]:
epochs = 25
batch = 25
history = model.fit(X_train, X_train, epochs=epochs, batch_size=batch, validation_split=.2, verbose=1).history

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=[x for x in range(len(history['loss']))], y=history['loss'], mode='lines', name='loss'))

fig.add_trace(go.Scatter(x=[x for x in range(len(history['val_loss']))], y=history['val_loss'], mode='lines', name='validation loss'))

fig.update_layout(title="LSTM AE Error loss over epochs", yaxis=dict(title="Loss"), xaxis=dict(title="Epoch"))

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=[x for x in range(len(history['accuracy']))], y=history['accuracy'], mode='lines', name='accuracy'))

fig.add_trace(go.Scatter(x=[x for x in range(len(history['val_accuracy']))], y=history['val_accuracy'], mode='lines', name='validation accuracy'))

fig.update_layout(title="LSTM AE Accuracy over epochs", yaxis=dict(title="Loss"), xaxis=dict(title="Epoch"))

fig.show()

In [ ]:
# Check how loss & mse went down
epoch_loss = history['loss']
epoch_val_loss = history['val_loss']
epoch_mae = history['accuracy']
epoch_val_mae = history['val_accuracy']

plt.figure(figsize=(8,5))
plt.plot(range(0,len(epoch_loss)), epoch_loss, 'b-', linewidth=2, label='Train Loss')
plt.plot(range(0,len(epoch_val_loss)), epoch_val_loss, 'r-', linewidth=2, label='Test Loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")

#lt.title('Loss')
plt.legend(loc='best')
plt.savefig('Figure_Loss_LSTM_AE_Load_Power.jpeg')
plt.show()

In [ ]:
# Check how loss & mse went down
epoch_loss = history['loss']
epoch_val_loss = history['val_loss']
epoch_mae = history['accuracy']
epoch_val_mae = history['val_accuracy']

plt.figure(figsize=(8,5))

plt.plot(range(0,len(epoch_mae)), epoch_mae, 'b-', linewidth=2, label='Train Acc')
plt.plot(range(0,len(epoch_val_mae)), epoch_val_mae, 'r-', linewidth=2,label='Test Acc')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
#plt.title('Accuracy')
plt.legend(loc='lower right')

plt.savefig('Figure_Acc_LSTM_AE_Load_Power.jpeg')
plt.show()

# Saving The Model

In [ ]:
# Save the model and architecture to single file
model.save('Diesel_Engine-LSTM_AE_Load_Power_model.h5')
print("Model Saved to a Disk")